In [32]:
from collections import OrderedDict
from datasets import mnist
from percep import softmax


class MuLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

In [33]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MuLayer()
mul_tax_layer = MuLayer()

price = mul_tax_layer.forward(mul_apple_layer.forward(apple, apple_num), tax)
price

220.00000000000003

In [34]:
dprice = 1
dapple_num = 2

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [35]:
class AddLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y

        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [36]:
apple = 100
apple_num = 2

mandarin = 150
mandarin_num = 3

tax = 1.1

layer1_1 = MuLayer()
out1_1 = layer1_1.forward(apple, apple_num)

layer1_2 = MuLayer()
out1_2 = layer1_2.forward(mandarin, mandarin_num)

layer2 = AddLayer()
out2 = layer2.forward(out1_1, out1_2)

layer3 = MuLayer()
out3 = layer3.forward(out2, tax)

out3

715.0000000000001

In [37]:
dprice1 = 1

dsummation, dtax = layer3.backward(dprice1)
dapple_price, dmandarin_price = layer2.backward(dsummation)
dapple, dapple_num = layer1_1.backward(dapple_price)
dmandarin, dmandarin_num = layer1_2.backward(dmandarin_price)

print(dapple_num, dapple, dmandarin, dmandarin_num, dtax)

110.00000000000001 2.2 3.3000000000000003 165.0 650


In [38]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0 # if self.mask is true, then fill indexes with zero

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [39]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [40]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

In [41]:
a1 = np.array([[1,1], [1,1]])
a2 = np.array([[2,3,4], [3,6,5]])

np.dot(a1, a2)

X = np.random.rand(2)
X

array([0.79894766, 0.10196879])

In [42]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

In [46]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    print(f"x shape: {x.shape}")
    for idx in np.ndindex(x.shape):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)

        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = tmp_val

    return grad  # 원래 모양의 gradient 반환

In [51]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    _batch_size = y.shape[0]
    delta = 1e-7
    return -np.sum(t * np.log(y + delta)) / _batch_size

In [60]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

In [63]:
from collections import OrderedDict

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Affine'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1) # 행에서 가장큰 원소의 인덱스
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads

    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()

        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine'].dW
        grads['b1'] = self.layers['Affine'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

In [64]:
from datasets.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(f"{key} : {str(diff)}")

x shape: (784, 50)
x shape: (50,)
x shape: (50, 10)
x shape: (10,)
W1 : 9.784482479332051e-05
b1 : 0.0007581490202363057
W2 : 0.0024338437850680737
b2 : 0.06666666678679281
